# StressME for wild type strain

In [98]:
import cobrame
import ecolime
import AcidifyME
import pickle
from AcidifyME import *
from AcidifyME.periplasmic_proteome import *
from AcidifyME.membrane_lipid_constraint import *
from AcidifyME.membrane_protein_activity import *
from AcidifyME.proton_influx import *
from oxidizeme.model import *

from qminospy.me1 import ME_NLP1
from cobrame.io.json import load_json_me_model

### To load wild type ecolime as json

In [100]:
me = load_json_me_model('EcoliME_wildtype.json')

### To define stress conditions

In [101]:
temperature = 37
ROS = 1.0 # fold
pH = 7.0

### To apply thermal stress

In [102]:
ecolime.chaperones.add_foldme_module(me)
ecolime.chaperones.change_temperature(me, temperature)

### To apply acid stress

In [103]:
fold_change=5.666666666669649*pH**4-1.396666666667337e+02*pH**3+1.286583333333885e+03*pH**2-5.254083333335287e+03*pH**1+8.037000000002481e+03
print(fold_change)

#add different constraints into the ME-model to simulate acid stress
temperatureK=273.15+temperature

add_membrane_constraint(me, 'MJR_habituated')
add_protein_stability_constraint(me, pH, temperatureK, constrain_periplasm_biomass=False)
add_proton_leak_rxn(me, pH)
modify_membrane_protein_activity(me, fold_change)

1.0


/opt/conda/envs/cobrame/user_packages/cobrame/cobrame/core/reaction.py:852 UserWarning: RNA Polymerase () not found
/opt/conda/envs/cobrame/user_packages/cobrame/cobrame/core/processdata.py:858 UserWarning: RNA_b3509 starts with 'AUG' which is not a start codon
/opt/conda/envs/cobrame/user_packages/cobrame/cobrame/core/processdata.py:896 UserWarning: No termination enzyme for RNA_b3509


### To apply oxidative stress

In [105]:
solver = ME_NLP1(me)

In [106]:
stress = StressME(solver)
stress.make_stressme()

10fthf_c + 1.24874498849701e-7*mu Fmt_mono_mod_mg2_mod_k + 0.00263713786673296*mu FusA_mono + 2.81444809683347e-6*mu InfA_mono + 2.93932259568317e-6*mu InfB_mono + 2.81444809683347e-6*mu InfC_mono + 1.43296649075618e-7*mu Map_mono_mod_2:fe2 + 1.13311230699714e-7*mu PrfB_mono + 2.81444809683347e-6*mu PrfC_mono + 0.000498399634202103*mu + 0.000195123456790123 + 0.00598079561042524*(mu + 0.3915)/mu RNA_b0026 + 2.55589556001079e-8*mu + 1.00063311174422e-8 RNA_degradosome + 1.80451491063336e-7*mu Rrf_mono + 0.00263713786673296*mu Tsf_mono + 92 ala__L_c + 46 arg__L_c + 21 asn__L_c + 71 asp__L_c + 937.0 + 4.21048010973937*(mu + 0.3915)/mu atp_c + 14 cys__L_c + 0.00263713786673296*mu generic_Tuf + 48.0 generic_tRNA_AAA_lys__L_c + 18.0 generic_tRNA_AAC_asn__L_c + 13.0 generic_tRNA_AAG_lys__L_c + 3.0 generic_tRNA_AAU_asn__L_c + 2.0 generic_tRNA_ACA_thr__L_c + 32.0 generic_tRNA_ACC_thr__L_c + 6.0 generic_tRNA_ACG_thr__L_c + 7.0 generic_tRNA_ACU_thr__L_c + 9.0 generic_tRNA_AGC_ser__L_c + 4.0 gener

In [107]:
subs_dict = {
 'h2o2_c': 5e-08, 'o2s_c': 2e-10,
 #'h2o2_e': 1.5e-06, 'o2s_e': 0,
 #'pq2_c': 0.0
}

subs_dict['h2o2_c'] = 5e-08*ROS
subs_dict['o2s_c'] = 2e-10*ROS

stress.substitute_ros(solver, subs_dict)
stress.substitute_metal(solver)

In [112]:
#### Unlock ROS detox enzymes, as they might be blocked (UB=0) for some reason by default
for r in me.process_data.SPODM.parent_reactions:
    print(r.id, '\t', r.upper_bound)
    r.upper_bound = 1000
    print(r.id, '\t', r.upper_bound)
    
for r in me.process_data.SPODMpp.parent_reactions:
    print(r.id, '\t', r.upper_bound)
    r.upper_bound = 1000
    print(r.id, '\t', r.upper_bound)
    
for r in me.process_data.CAT.parent_reactions:
    print(r.id, '\t', r.upper_bound)
    r.upper_bound = 1000
    print(r.id, '\t', r.upper_bound)
    
me.reactions.MOX_REV_CPLX_dummy.upper_bound = 0

SPODM_FWD_SUPEROX-DISMUTMN-CPLX_mod_mn2 	 0
SPODM_FWD_SUPEROX-DISMUTMN-CPLX_mod_mn2 	 1000
SPODM_FWD_SUPEROX-DISMUTFE-CPLX_mod_fe3 	 0
SPODM_FWD_SUPEROX-DISMUTFE-CPLX_mod_fe3 	 1000
SPODMpp_FWD_G6886-MONOMER_mod_cu2 	 0
SPODMpp_FWD_G6886-MONOMER_mod_cu2 	 1000
CAT_FWD_HYDROPEROXIDII-CPLX_mod_6:hemed 	 0
CAT_FWD_HYDROPEROXIDII-CPLX_mod_6:hemed 	 1000
CAT_FWD_HYDROPEROXIDI-CPLX_mod_pheme 	 0
CAT_FWD_HYDROPEROXIDI-CPLX_mod_pheme 	 1000


In [113]:
stress.force_o2s_from_pq()

for r in me.reactions.query('PQ2RED'):
    print(r.id, '\t', r.lower_bound)

PQ2RED_FWD_FLAVONADPREDUCT-MONOMER_mod_fad 	 0.3206684


### To simulate under stress conditions

In [114]:
sol = solver.bisectmu(1e-4)

 Allocating name1, name2
 Allocating pi, rc, xn
 Allocated pi, rc, xn
 No Jacobian matrix since no nonlinear obj or constraints
BFFF0000000000000000000000000000
 BFF0000000000000
BFFF0000000000000000000000000000
C083D6329F1C35CA5000000000000000
 C83D6329F1C35CA5
C083D6329F1C35CA5000000000000000
4083D6329F1C35CA5000000000000000
 483D6329F1C35CA5
4083D6329F1C35CA5000000000000000
 
      quadMINOS  5.6      (Jul 2016)
 Calling miopt to set option: Maximize
 Calling miopt to set option: Solution No
 Calling miopti to set option New basis file to           11
 Calling miopti to set option Save frequency to       500000
 Calling miopti to set option Print level to            0
 Calling miopti to set option Print frequency to       100000
 Calling miopti to set option Scale option to            2
 Calling miopti to set option Iteration limit to      2000000
 Calling miopti to set option Expand frequency to       100000
 46293E5939A08CEA
406293E5939A08CEA000000000000000
 Calling mioptr to set 

### Results and save as sol.pickle

In [ ]:
# solution status
print(me.solution.status)
# growth rate
print(me.reactions.biomass_dilution.x)
# to save solution to pickle
# write solution file 
filename = 'StressME_wildtype'+'_T_'+str(temperature)+'_pH_'+str(pH)+'_ROS_'+str(ROS)+'X'
with open(filename+'_sol.pickle','wb') as f:
    pickle.dump(me.solution,f)